In [11]:
import torch
import torch.autograd as autograd
import numpy as np
import matplotlib.pyplot as plt

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

In [12]:
!pip install pyro-ppl

In [13]:
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
import torch.nn as nn

from pyro.infer import MCMC, NUTS, Predictive

In [14]:
# Real Observations

t_obs = torch.linspace(0., 5., 50).unsqueeze(-1)          # 50 × 1
u_true = 0.5*t_obs**2 + t_obs
noise = 0.1
u_obs = u_true + noise * torch.randn_like(u_true)     # 50 X 1
u_obs = u_obs.squeeze()


# Collocation Points
t_f = torch.linspace(0., 5., 70).unsqueeze(-1).requires_grad_(True)

# initial condition point
t_ic = torch.tensor([[0.0]])

In [15]:
class PhyBNN(PyroModule):
    def __init__(self, in_dim=1, out_dim=1, hid_dim=50, prior_scale=10.):
        super().__init__()

        self.activation = nn.Tanh()  
        self.layer1 = PyroModule[nn.Linear](in_dim, hid_dim)  # Input to hidden layer
        self.layer2 = PyroModule[nn.Linear](hid_dim, hid_dim)
        self.layer3 = PyroModule[nn.Linear](hid_dim, out_dim)  # Hidden to output layer

        # Set layer parameters as random variables
        self.layer1.weight = PyroSample(dist.Normal(0., prior_scale).expand([hid_dim, in_dim]).to_event(2))
        self.layer1.bias = PyroSample(dist.Normal(0., prior_scale).expand([hid_dim]).to_event(1))
        self.layer2.weight = PyroSample(dist.Normal(0., prior_scale).expand([hid_dim, hid_dim]).to_event(2))
        self.layer2.bias = PyroSample(dist.Normal(0., prior_scale).expand([hid_dim]).to_event(1))
        self.layer3.weight = PyroSample(dist.Normal(0., prior_scale).expand([out_dim, hid_dim]).to_event(2))
        self.layer3.bias = PyroSample(dist.Normal(0., prior_scale).expand([out_dim]).to_event(1))

    
    def neural_net(self,t):
        h_1 = self.activation(self.layer1(t))
        h_2 = self.activation(self.layer2(h_1))
        return self.layer3(h_2).squeeze(-1)
    

    def forward(self,t_f = None,t_ic = None):
        #t_f = t_f.requires_grad_(True)
        # u_pred_obs = self.neural_net(t_obs)
        # u_ic = self.neural_net(t_ic)
        # u_f = self.neural_net(t_f)

        # du_dt = autograd.grad(u_f,t_f,grad_outputs=torch.ones_like(u_f),create_graph=True)[0].squeeze(-1)
        # res_pde =  du_dt - t_f.squeeze(-1)

        # res_ic = u_ic - 0.0
        

    # def forward(self, t_obs, u_obs=None):
    #     t = t_obs.reshape(-1, 1)
    #     h_1 = self.activation(self.layer1(t))
    #     h_2 = self.activation(self.layer2(h_1))
    #     u_pred_obs = self.layer3(h_2).squeeze()

        sigma = pyro.sample("sigma", dist.Gamma(.5, 1))  # Infer the response noise
        
    

        # if u_obs is not None:
        #     with pyro.plate("data", t_obs.shape[0]):
        #         pyro.sample("obs", dist.Normal(u_pred_obs, sigma * sigma), obs=u_obs)

        if t_f is not None:
            t_f = t_f.requires_grad_(True)
            u_pred_f = self.neural_net(t_f).squeeze(-1)
            du_dt = autograd.grad(u_pred_f,t_f,grad_outputs=torch.ones_like(u_pred_f),create_graph=True)[0].squeeze(-1)
            res_pde =  du_dt - t_f.squeeze(-1)
            with pyro.plate("physics",t_f.shape[0]):
                pyro.sample("pde", dist.Normal(res_pde, sigma * sigma), obs=torch.zeros_like(res_pde))

        if t_ic is not None:
            u_pred_ic = self.neural_net(t_ic).squeeze(-1)
            res_ic = u_pred_ic - 0.0
            with pyro.plate("initial",t_ic.shape[0]):
                pyro.sample("ic", dist.Normal(res_ic, sigma * sigma), obs=torch.zeros_like(res_ic))




    #     # Data Likelihood
        # with pyro.plate("data", t_obs.shape[0]):
        #     pyro.sample("obs", dist.Normal(u_pred_obs, sigma * sigma), obs=u_obs)
        #       #obs = pyro.sample("obs", dist.Normal(u_pred_obs, sigma * sigma), obs=u_obs)

        # IC Likelihood
        # with pyro.plate("initial",t_ic.shape[0]):
        #     pyro.sample("ic", dist.Normal(res_ic, sigma * sigma), obs=torch.zeros_like(res_ic))

        # PDE Likelihood
        # with pyro.plate("physics",t_f.shape[0]):
        #     pyro.sample("pde", dist.Normal(res_pde, sigma * sigma), obs=torch.zeros_like(res_pde))


        #return u_pred_f if t_f is not None else u_pred_obs

        if t_f is not None:
            return u_pred_f
        if t_ic is not None:
            return u_pred_ic
        return None

In [16]:
model_phy = PhyBNN()

pyro.set_rng_seed(42)

nuts_kernel = NUTS(model_phy, jit_compile=False)
mcmc = MCMC(nuts_kernel, num_samples=100, warmup_steps=50, num_chains=1)

with torch.enable_grad():
    mcmc.run(t_f, t_ic)



#mcmc.run(t_obs,u_obs,t_f, t_ic)

posterior_samples = mcmc.get_samples()

Sample: 100%|██████████| 150/150 [02:38,  1.05s/it, step size=6.73e-04, acc. prob=0.382]


In [17]:
for name, vals in posterior_samples.items():
    print(name, vals.shape)

layer1.bias torch.Size([100, 50])
layer1.weight torch.Size([100, 50, 1])
layer2.bias torch.Size([100, 50])
layer2.weight torch.Size([100, 50, 50])
layer3.bias torch.Size([100, 1])
layer3.weight torch.Size([100, 1, 50])
sigma torch.Size([100])


In [21]:
# Stage 2: Condition on data, starting from the physics‐informed posterior


mean_w1 = posterior_samples["layer1.weight"].mean(0)
std_w1  = posterior_samples["layer1.weight"].std(0)

print(mean_w1.shape)

mean_b1 = posterior_samples["layer1.bias"].mean(0)
std_b1 = posterior_samples["layer1.bias"].std(0)

print(mean_b1.shape)





mean_w2 = posterior_samples["layer2.weight"].mean(0)
std_w2  = posterior_samples["layer2.weight"].std(0)

mean_b2 = posterior_samples["layer2.bias"].mean(0)
std_b2 = posterior_samples["layer2.bias"].std(0)





mean_w3 = posterior_samples["layer3.weight"].mean(0)
std_w3  = posterior_samples["layer3.weight"].std(0)

mean_b3 = posterior_samples["layer3.bias"].mean(0)
std_b3 = posterior_samples["layer3.bias"].std(0)

torch.Size([50, 1])
torch.Size([50])
